# **Methanol Synthesis by Catalytic Hydrogenation of CO2**
<br>

## **1. Introduction**

This script aims to simulate a catalytic reactor design to perform the synthesis of methanol by hydrogenation of CO2. This simulations is based on the work of Gosh *et al.*, 2021.



## **2. Hypothesis about the system**

- gaseous phase is ideal
- stationary operation
- reactions follow a Langmuir-Hinshelwood mechanism, where the reactant must adsorve over the catalyst surface before reaction
- reference state is 25 °C and 1 bar
- methanol adsorption obver catalyst is negligible
- both single site and dual site approaches are tested
- enthalpy of adsorption is constant over the temperature range
- pressure in the bed length is given by Ergun's equation (only for PBR approach)
- mass transfer resistances are negligible
- no radial dispersion (PBR approach)
- the catalyst suffers no desactivation over time
- both isothermal and adiabatic approaches are tested
- a $In_2O_3$ catalyst is applied

## **3. Reactions considered in this system**

<br>

$$CO_{2(g)} + 3 H_{2(g)}\rightleftharpoons CH_3OH_{(g)} + H_2O_{(g)}$$

$$CO_{2(g)} + H_{2(g)} \rightleftharpoons CO_{(g)} + H_2O_{(g)}$$

$$CO_{2(g)} + 4 H_{2(g)}\rightleftharpoons CH_4{(g)} + 2 H_2O_{(g)}$$


## **4. Rate laws for Langmuir-Hinshelwood mechanism**

By following the Langmuir-Hinshelwood mechanism, the rate laws are given by equations below:

$$r_{CH_3OH}=\frac{k_1 \frac{\Bigg(P_{CO_2}P_{H_2}^3 - \frac{P_{CH_3OH}P_{H_2O}}{K_{eq, CH_3OH}}\Bigg)}{P_{H_2}^2}}{\text{Inhibition Term}}$$

<br>

$$r_{RWGS} = \frac{k_2 \frac{\Bigg(P_{CO_2}P_{H_2} - \frac{P_{CO} P_{H_2O}}{K_{eq, RWGS}}\Bigg)}{\sqrt{P_{H_2}}}}{\text{Inhibition Term}}$$

<br>

$$r_{CH4} = k_3\sqrt{P_{CO_2}} \sqrt{P_{H2}}\frac{\Bigg(1-\frac{P_{CH_4}P_{H_2O}^2}{P_{CO_2}P_{H_2}^4 K_{eq, CH_4}}\Bigg)}{\text{Inhibition Term}}$$

<br>

The expression for the inhibition term will depend on the approach, as follows:

<br>

$$\text{Inhibition Term} = (1+K_{CO_2}P_{CO_2}+\sqrt{K_{H2}P_{H2}})^2\text{ for the single-site approach}$$

<br>

$$\text{Inhibition Term} = (1+K_{CO_2}P_{CO_2})(1+\sqrt{K_{H2}P_{H2}})\text{ for the single-site approach}$$

<br>

Where: 
- $K_{eq,i}$ is the chemical equilibrium constant, calculated by Gibbs Free Energy, for the reaction $i$; 
- $K_{ads, k}$ is adsorption equilibrium constant for the component $k$ over the catalyst surface; 
- $k_i$ is the reaction constant, calculated by Arrhenius law for the reaction $i$, 
- $P_k$ is the partial pressure of the component $k$


## **5. Import libraries and define global constants**

In [1]:
import numpy as np
import pandas as pd
import math as m
import warnings
import chemical_properties as cp
import thermodynamic as thermo
import kinetic as kn
import simulation as sim
import json

R = 8.314472                         # ideal gas universal constant (J/mol.K)

warnings.filterwarnings('ignore')

## **6. Import physical and chemical data**

In [2]:
# physical-chemical data stored as json file
f = open('data_dict.json')
data_dict = json.load(f)

# reaction data stored as json file
f = open('reaction_dict.json')
reaction_data = json.load(f)

## 7. Physical and chemical properties correlations

<br>

### 7.1 Ideal Gas Heat Capacity (J/kmol.K)

<br>

$$C_{P} = C_1+C_2\Bigg[\frac{\frac{C_3}{T}}{sinh\frac{C_3}{T}}\Bigg]^2+C_4\Bigg[\frac{\frac{C_5}{T}}{cosh \frac{C_5}{T}}\Bigg]^2$$

<br>

### 7.2. Vapor Viscosity (Pa.s)

<br>

$$\mu = \frac{C_1 T^{C_2}}{1+\frac{C_3}{T}+\frac{C_4}{T^2}}$$

<br>

### 7.3. Ideal Gas Mixture Vapor Viscosity (Pa.s)

<br>

$$\mu_{mix} = \sum_{i=1}^N \frac{y_i \mu_i}{\sum_{j=1}^N y_j \phi_{ij}}$$

<br>

$$\phi_{ij} = \frac{\Bigg[1+\sqrt{\frac{\mu_i}{\mu_j}}\Bigg(\frac{M_j}{M_i}\Bigg)^4\Bigg]^2}{\frac{4}{\sqrt{2}}\sqrt{1+\frac{M_i}{M_j}}}$$

<br>

### 7.4. Mixture Density (kg/m³)

<br>

$$\rho_{mix} = \frac{PM_{mix}}{RT}$$

<br>

$$M_{mix} = \sum_{i=1}^N y_i M_i$$

<br>

Where: $M_{mix}$ is the mixture molar mass and $y_i$ is the molar fraction of component $i$

## 8. Thermodynamic Functions - Gibbs Free Energy of Reaction and Equilibrium Constant

<br>

For a reversible reaction, the equilibrium constant is function only of temperature. For a specific reaction $k$:

<br>

$$K_{eq,k}=exp\Bigg(\frac{-\Delta g_{r,k}}{RT}\Bigg)$$

<br>

$$\Delta g_{r,k} = \Delta h_{r,k}-T \Delta s_{r,k}$$

$$\Delta h_{r,k} = \sum_{i=1}^N \nu_i \Delta h_i$$

$$\Delta h_i = \Delta h_{f,i}^0 + \int_{T_0}^T C_{P,i}(T) dT$$

$$\Delta s_{r,k} = \sum_{i=1}^N \nu_i \Delta s_i$$

$$\Delta s_i = \Delta s_{f,i}^0 + \int_{T_0}^T \frac{C_{P,i}(T)}{T} dT$$

<br>

Where: 
- $R$ is the ideal gas universal constant ($R = 8.314472 J/mol.K$); 
- $K_{eq,k}$ is the chemical equilibrium constant
- $\Delta g_{r,k}$ is the Gibbs Free Energy of reaction (J/mol)
- $\Delta h_{r,k}$ is the enthalpy of reaction (J/mol)
- $\Delta s_{r,k}$ is the entropy of reaction (J/mol.K)
- $T$ is the system's temperature (K)
- $\Delta h_i$ is the enthalpy of component $i$ at reaction's temperature (J/mol)
- $\Delta s_i$ is the entropy of component $i$ at reaction's temperature (J/mol.K)
- $\Delta h_{f,i}^0$ is the standard enthalpy of formation of component $i$ (J/mol)
- $\Delta s_{f,i}^0$ is the standard entropy of formation of component $i$ (J/mol.K)
- $\nu_i$ is the stoichiometric coefficient of component $i$
- $C_{P,i}(T)$ is the expression for the ideal gas heat capacity of component $i$ (J/mol.K)

## 9. Kinetic Functions - Reaction Constants and Adsorption Phenomena

<br>

$$k_i = A_i exp\Bigg(\frac{-E_{a,i}}{RT}\Bigg)$$

<br>

Where: 
- $A_i$ is the pre-exponential factor for the reaction $i$, 
- $E_{a,i}$ is activation energy for reaction $i$, 
- $R$ is the ideal gas universal constant (8.314472 J/mol.K) and $T$ is the temperature.

<br>

To model the adsorption phenomena, the Vant'Hoff equation was applied to model the effects of both pressure and temperature on the adsorption equilibria constants, as follows.

<br>

$$K_{ads}=K_{ref}\Bigg(exp\Bigg[\frac{\Delta h_{ads}}{R}\Bigg(\frac{1}{T_{ref}}-\frac{1}{T}\Bigg)\Bigg]\Bigg)$$

<br>

- $K_{ads}$ is the adsorption equilibrium constant at temperature $T$
- $K_{ref}$ is the adsorption equilibrium constant at reference temperature $T_{ref}$
- $\Delta h_{ads}$ is the heat of adsorption (J/mol)

In [3]:
data_dict

{'comp_list': ['CO2', 'H2', 'MeOH', 'H2O', 'CO', 'CH4'],
 'unit': {'molar_mass': 'kg/kmol',
  'IdealGasCp': 'J/kmol.K',
  'vapor_viscosity': 'Pa.s',
  'H0f': 'J/kmol',
  'S0f': 'J/kmol.K',
  'Kref_ads': '1/bar at 300 Â°C',
  'deltaHads': 'kJ/mol'},
 'molar_mass': {'CO2': 44.01,
  'H2': 2.02,
  'MeOH': 32.04,
  'H2O': 18.02,
  'CO': 28.01,
  'CH4': 16.04},
 'IdealGasCp': {'CO2': {'C1': 29370,
   'C2': 34540,
   'C3': 1428,
   'C4': 26400,
   'C5': 588},
  'H2': {'C1': 27617, 'C2': 9560, 'C3': 2466, 'C4': 3760, 'C5': 567.6},
  'MeOH': {'C1': 39252, 'C2': 87900, 'C3': 1916.5, 'C4': 53654, 'C5': 896.7},
  'H2O': {'C1': 33363, 'C2': 26790, 'C3': 2610.5, 'C4': 8896, 'C5': 1169},
  'CO': {'C1': 29108, 'C2': 8773, 'C3': 3085, 'C4': 8455.3, 'C5': 1538.2},
  'CH4': {'C1': 33298, 'C2': 79933, 'C3': 2086.9, 'C4': 41602, 'C5': 991.96}},
 'vapor_viscosity': {'CO2': {'C1': 2.148e-06, 'C2': 0.46, 'C3': 290, 'C4': 0},
  'H2': {'C1': 1.797e-07, 'C2': 0.685, 'C3': -0.59, 'C4': 140},
  'MeOH': {'C1': 3.06

In [4]:
reaction_data

{'unit': {'kref': 'mol/s.bar^n kg_cat', 'E_a': 'kJ/mol'},
 'description': {'1': 'CO2 + 3 H2 <---> CH3OH + H2O',
  '2': 'CO2 + H2 <---> CO + H2O',
  '3': 'CO2 + 4 H2 <---> CH4 + 2 H2O'},
 'stoichiometry': {'1': [-1, -3, 1, 1, 0, 0],
  '2': [-1, -1, 0, 1, 1, 0],
  '3': [-1, -4, 0, 2, 0, 1]},
 'kref': {'1': 0.00069, '2': 0.0018, '3': 0.00011},
 'E_a': {'1': 35.7, '2': 54.5, '3': 42.5},
 'equilibrium_constant': {'1': 1, '2': 2, '3': 3},
 'heat_reaction': {'1': 1, '2': 2, '3': 3}}

In [5]:
T0 = 200
P0 = 40
h2toco2 = 3
WHSV = 9000
reactor_setup = {
    'volume': 100,
    'LtoD': 1,
    'L': 1,
    'D':1
}

In [6]:
initial_cond, F_dict = sim.prepare_data(T0, P0, WHSV, reactor_setup, h2toco2, comp_dict = data_dict)

In [7]:
sim.energy_balance(data_dict, reaction_data, T0+273.15, P0, F_dict, initial_cond, single_model=False)

8.536838040424292e-06

In [8]:
sim.material_balance(T0+273.15, P0, F_dict, initial_cond, data_dict, reaction_data, single_model = True)

CO2    -0.000275
H2     -0.000734
MeOH    0.000227
H2O     0.000277
CO      0.000047
CH4     0.000002
Name: global_rate, dtype: float64

In [9]:
sim.reactor_dimensions(reactor_setup)
reactor_setup

{'volume': 100, 'LtoD': 1, 'L': 5.030795991604358, 'D': 5.030795991604358}

In [10]:
initial_cond[0]

143.59999999999994

In [11]:
m_in = 12*initial_cond[0]
sim.calculate_k(F_dict, data_dict, T0+273.15, P0, reactor_setup, m_in)

15.495052323387249
151.70868392282102
-42747.04967287222


-7147466.4188064905